# 05-思考模式 - DeepSeek API

deepseek-reasoner 深度推理示例

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../../.env')

api_key = os.getenv("VITE_DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("❌ 未找到 API Key")

client = OpenAI(
    api_key=api_key,
    base_url=os.getenv("VITE_DEEPSEEK_BASE_URL", "https://api.deepseek.com"),
)

## deepseek-reasoner（强制思考）

In [ ]:
# 使用 deepseek-reasoner
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
)

message = response.choices[0].message

# 访问思维链
if hasattr(message, 'reasoning_content'):
    print("🧠 思考过程：")
    print("=" * 60)
    print(message.reasoning_content)
    print("=" * 60)

print("\n📄 最终答案：")
print(message.content)

## 思考模式 + 流式输出

In [ ]:
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[{"role": "user", "content": "证明勾股定理"}],
    stream=True
)

print("🧠 思考过程：")
print("-" * 60)

in_content = False
for chunk in response:
    delta = chunk.choices[0].delta
    
    if hasattr(delta, 'reasoning_content') and delta.reasoning_content:
        print(delta.reasoning_content, end="", flush=True)
    
    if delta.content:
        if not in_content:
            in_content = True
            print("\n\n📄 最终答案：")
            print("-" * 60)
        print(delta.content, end="", flush=True)

print()